# FIT5196 Task 1 in Assessment 3
#### Student Name: Sai Ram D Gitte
#### Student ID: 31009751

Date: 21/11/2020

Version: 1.0

Environment: Python 3.7.4 and Jupyter notebook


## Task 1 - Data Integration

#### Introduction:
When someone is looking out to rent/buy a property, he/she looks at different aspects in and around the property to decide if they would want to live there. Collecting all the possible data for a property streamlines the process of narrowing down a property. Few of the aspects one would look into is if the property is located around basic necessities and amenities like:

1. Supermarkets
2. Shopping centers
3. Hospitals
4. Train stations

Now, to extend the information provided for each property, the final result would also include details on which are the nearest supermarkets, shopping centers, hospitals and train stations, whats the average time taken to reach the City (i.e., Flinders Station) through train from the station that is closely situated to each of the properties. If there are direct trains 

Task 1 focuses on integarting data from different sources such as pdfs, txt, xml, json, xlsx, html and shp files into once csv file. Along with this, there are a few logical computations to be done in order to arrive at the desired columns and their values.    
    

In [ ]:
# Install pdfminer 
!pip install pdfminer.six


In [ ]:

import pandas as pd
import numpy as np
from pdfminer.high_level import extract_text


# Supermarket
* To read the pdf I have used the 'pdfminer.six' package. In that, the module named 'extract_text' helps us to read the pdf file into a variable. 
* The read text is then run inside a loop, splitting the entire content on a newline '\n' and the appending it to a list called 'data'. 
* A try and except block is used to check if the data that is split has a number as its first elemnet, if thats true, then append it to the list else do not do anything and go back to the loop

In [ ]:
# read supermarket pdf file
text = extract_text('supermarkets.pdf') # Reading the pdf data into a variable

data = [] #emply list 
for i in text.split('\n'): # Looping through the split data
    if i != '':
        try:
            if int(i[0]) or i[0] == '0':
                data.append(i.split(' ')) #updating the empty list 
        except:
            pass


In [ ]:
# Converting read pdf into a dataframe

df_supermarket = pd.DataFrame(data, columns=['dummy', 'id','lat','lng', 'type']) 

df_supermarket.drop(['dummy'], axis = 1, inplace = True) # Dropping the 'dummy' column from the dataframe

In [ ]:
df_supermarket

# Hospital id
* To read the data into pandas dataframe, I have used the read_excel function within pandas
* This will read the data into a dataframe format 

In [ ]:
# # Installing optional dependency xlrd
# !pip install xlrd


In [ ]:
df_hospital = pd.read_excel('hospitals.xlsx', index_col=0)
df_hospital

# Shopping Center
* To read the 'html' file into a dataframe, pandas.read_html() is used. 
* pd.read_html returns the read dataframe into list of dataframes. 
* Here there is only one element in the list of dataframe and hence 'df_shopping[0]' is used to retrieve the dataframe
* Drop the unwanted column

In [ ]:

df_shopping = pd.read_html('shopingcenters.html') # Reading the html file into a dataframe
df_shopping = df_shopping[0] # 
df_shopping.drop('Unnamed: 0', axis = 1, inplace = True)
df_shopping


# Real estate
## There are two types of file here: 'json' and 'xml':
        To read the json file:
            1. Use the pandas.read_json() function which will directly read the json contents to a dataframe
        
        To read the xml file:
        
            1. Read it using the 'open()' function in read mode and passing the encoding as 'utf-8'

                - this returns a single string

            2. Eliminate the first and last character of the above string (unwanted characters)
            3. Create a list of list which contains all the column values in it separately as different lists
            4. Converting this lists of list into a dataframe with 10 columns

In [ ]:
df_real = pd.read_json("real_state.json") # Reading the json file

In [ ]:
file = open("real_state.xml", 'r', encoding = 'utf8') 
xml_str = file.readlines()[0][2:-1] # Reading the xml file into a string
xml_str

In [ ]:
# Convert 
import xml.etree.ElementTree as ET

root = ET.fromstring(xml_str) # Root has the data in a dictionary format

main_list = [] # Empty list 
for i in range(len(root)): # Looping through the dictionary
    sub_list = []
    for j in range(len(root[i])):
        sub_list.append(root[i][j].text) # Extract the values from the dictionary and update the list
    main_list.append(sub_list) 


In [ ]:
# Check the column names
for i in root:
    print(i)

In [ ]:
# Create a list of column names
colnames = ['property_id', 'lat', 'lng', 'addr_street', 'price', 'property_type', 'year', 'bedrooms', 'bathrooms', 'parking_space']

# Create a pandas dataframe
df_xml = pd.DataFrame(columns = colnames)

# Assign the each list in the lists of list to the dataframe columns 
for i in range(len(root)):
    df_xml[colnames[i]] = main_list[i]
df_xml

# GTFS
* Reading just the required files from GTFS to start working on nearest train station ids, travel min to CBD and Transfer flag

In [ ]:
# Reading the Trips.txt file
df_gtfs_trips = pd.read_csv('trips.txt', sep=",", header=None)

# Defining the column names
df_gtfs_trips.columns = ["route_id", "service_id", "trip_id", "shape_id","trip_headsign", "direction_id"]

# Except the the first row, retain everything
df_gtfs_trips = df_gtfs_trips.iloc[1:] 
df_gtfs_trips

In [ ]:
# Reading the Stops.txt file
df_gtfs_stops = pd.read_csv('stops.txt', sep=",", header=None)

# Defining the column names
df_gtfs_stops.columns = ["stop_id", "stop_name", "stop_short_name", "stop_lat","stop_lon"]

# Except the the first row, retain everything
df_gtfs_stops = df_gtfs_stops.iloc[1:]
df_gtfs_stops

In [ ]:
# Reading the Stop_times.txt file
df_gtfs_stop_times = pd.read_csv('stop_times.txt', sep=",", header=None)

# Defining the column names
df_gtfs_stop_times.columns = ["trip_id", "arrival_time", "departure_time", "stop_id","stop_sequence","stop_headsign","pickup_type","drop_off_type","shape_dist_traveled"]

# Except the the first row, retain everything
df_gtfs_stop_times = df_gtfs_stop_times.iloc[1:]
df_gtfs_stop_times

In [ ]:
# Cleaning the departure_time column
import re

df_gtfs_stop_times_dup = df_gtfs_stop_times.copy()

for item, row in df_gtfs_stop_times_dup.iterrows():
    if re.match(r'^24', row['departure_time'].split(':')[0]):
        df_gtfs_stop_times.loc[item, 'departure_time'] = re.findall(r'24', df_gtfs_stop_times_dup.loc[item,'departure_time'])[0].replace(re.findall(r'24', df_gtfs_stop_times_dup.loc[item,'departure_time'])[0], '00:00:00')
        
    if re.match(r'^25', row['departure_time'].split(':')[0]):
        df_gtfs_stop_times.loc[item, 'departure_time'] = re.findall(r'25', df_gtfs_stop_times_dup.loc[item,'departure_time'])[0].replace(re.findall(r'25', df_gtfs_stop_times_dup.loc[item,'departure_time'])[0], '00:00:00')
        
    if re.match(r'^26', row['departure_time'].split(':')[0]):
        df_gtfs_stop_times.loc[item, 'departure_time'] = re.findall(r'26', df_gtfs_stop_times_dup.loc[item,'departure_time'])[0].replace(re.findall(r'26', df_gtfs_stop_times_dup.loc[item,'departure_time'])[0], '00:00:00')
     

# Final dataframe
* Combine json and xml converted dataframe into one dataframe
* Drop any duplicate rows that are present after combing the dataframes
    - 7 rows were dropped

In [ ]:
# final df
df_final = pd.concat([df_real, df_xml])
df_final

In [ ]:
df_final = df_final.drop_duplicates()
df_final = df_final.reset_index()

# Haversine function
- Defining a function to calculate the haversine distance between two coordinates

In [ ]:
# Function to calculate the distance between customer loc to the nearest warehouse
from math import *

def haversine_fun(latitude1, longitude1, latitude2, longitude2):
    # Given that the radius of the earth should be taken 6378 km 
    earth_radius = 6378 
    
    # Take the difference of the two latitudes and convert them to radians
    lat_diff = radians(latitude2 - latitude1) 
    
    # Take the difference of the two Longitudes and convert them to radians
    lon_diff = radians(longitude2 - longitude1) 
    
    # Convert the first latitude to radians
    latitude1 = radians(latitude1) 
    
    # Convert the second latitude to radians
    latitude2 = radians(latitude2) 

    x = sin(lat_diff/2)**2 + cos(latitude1)*cos(latitude2)*sin(lon_diff/2)**2 
    y = 2*asin(sqrt(x))

    return earth_radius * y # Returns the distance between two coordinates 

# Refer: From Assignment-2

# Distance shopping center
- Converting the latitude and longitude values to float datatype
- Calculate distance of each property location with each of the shopping centers
- Then calculate which is the minimum distance and its indices
- Loop through df_shopping dataframe and use the above mentioned indices to fetch the 'sc_id' of the shopping center
- Update the final dataframe with the new values found

In [ ]:
# Converting the latitude and longitude values to float from string
df_final['lat'] = df_final.lat.astype(float)
df_final['lng'] = df_final.lng.astype(float)

df_shopping['lat'] = df_shopping.lat.astype(float)
df_shopping['lng'] = df_shopping.lng.astype(float)

In [ ]:
# Calculating all the distances from the shopping centre to the property 
main_dist_list = []
for item, row in df_final.iterrows(): # Looping through the final dataframe
    dist_list = []
    for item1, row1 in df_shopping.iterrows():
        
        # Calculating the distance between 2 coordinates
        dist_list.append(haversine_fun(
            row['lat'],row['lng'], 
            row1['lat'], row1['lng']))
        # Update the main list
    main_dist_list.append(dist_list)
main_dist_list # List of lists

In [ ]:
# Calculating the minumum distance from property to shopping center
new = [] 

# List contains the minimum distance for all the rows
min_dist = [] 
for i in main_dist_list:
    new.append(i.index(min(i)))
    
    # Rounding off the distance to 4 decimals
    min_dist.append(round(min(i),4)) 
column_name = np.array(new) 

# Creating a new column which will contain the 
# corresponding indices of shopping_centers names
df_final['indexo'] = column_name 
df_final

In [ ]:
# Closest shopping center
shopping_center_id = []

# Looping through the final df and shopping df to retrieve the shopping center id
for item, row in df_final.iterrows():
    
    shopping_center_id.append(df_shopping.loc[row['indexo'], 'sc_id'])

# Update the final df with the shopping center id

df_final['Shopping_center_id'] = shopping_center_id

# Update the final df with the minimum distance to shopping center
df_final['Distance_to_sc'] = min_dist
df_final

In [ ]:
df_final = df_final.drop(['index', 'indexo'], axis = 1)

# Station ID
- Converting the latitude and longitude values to float datatype
- Calculate distance of each property location with each of the train centers
- Then calculate which is the minimum distance and its indices
- Loop through df_gtfs_stops dataframe and use the above mentioned indices to fetch the 'Train_station_id'
- Update the final dataframe with the new values found

In [ ]:
# convert the latitude and longitude 
df_gtfs_stops['stop_lat'] = df_gtfs_stops.stop_lat.astype(float)

df_gtfs_stops['stop_lon'] = df_gtfs_stops.stop_lon.astype(float)

In [ ]:
# List of lists of distances
main_dist_list = []
for item, row in df_final.iterrows():
    dist_list = []
    # Loop through the stops file
    for item1, row1 in df_gtfs_stops.iterrows(): 
        
        # Calculate the distance between 
        # train_station and property
        dist_list.append(haversine_fun(
            row['lat'],row['lng'], 
            row1['stop_lat'], row1['stop_lon']))
    main_dist_list.append(dist_list)
main_dist_list

In [ ]:
new = [] 

# List contains the minimum distance for all the rows
min_dist = [] 
for i in main_dist_list:
    new.append(i.index(min(i)))
    
    # Rounding off the distance to 4 decimals
    min_dist.append(round(min(i),4)) 
column_name = np.array(new) 

# Creating a new column which will 
# contain the corresponding indices of station_id
df_final['dist_index'] = column_name 
df_final

In [ ]:
# Closest station
train_station_id = []

# Loop through the final dataframe
for item, row in df_final.iterrows():
    
    # Update the list with the corresponding station ids
    train_station_id.append(df_gtfs_stops.loc[row['dist_index'], 'stop_id'])

# Updating the train station id in the 
# final dataframe
df_final['Train_station_id'] = train_station_id

# Updating the train station dist in the 
# final dataframe
df_final['Distance_to_train_station'] = min_dist
df_final


In [ ]:
df_final = df_final.drop(['dist_index'], axis = 1)
df_final 

# Distance to hospital
- Converting the latitude and longitude values to float datatype
- Calculate distance of each property location with each of the hospitals
- Then calculate which is the minimum distance and its indices
- Loop through df_hospitals dataframe and use the above mentioned indices to fetch the 'hospital_id'
- Update the final dataframe with the new values found

In [ ]:
df_hospital

In [ ]:
# Converting the string latitude and longitude values
# to float datatype

df_hospital['lat'] = df_hospital.lat.astype(float)
df_hospital['lng'] = df_hospital.lng.astype(float)

In [ ]:
# List of lists of distances
main_dist_list = []

# Loop through the final dataframe to calculate the 
# distances from property to hospitals
for item, row in df_final.iterrows():
    dist_list = []
    for item1, row1 in df_hospital.iterrows():
        dist_list.append(haversine_fun(
            row['lat'],row['lng'], 
            row1['lat'], row1['lng']))
    main_dist_list.append(dist_list)
main_dist_list

In [ ]:
# Calculating the minimum distance from nearest hospital
# to the property

new = [] 

# List contains the minimum distance for all the rows
min_dist = [] 
for i in main_dist_list:
    new.append(i.index(min(i)))
    
    # Rounding off the distance to 4 decimals
    min_dist.append(round(min(i),4)) 
column_name = np.array(new) 

# Creating a new column which will contain the 
# corresponding indices of hospital ids
df_final['hospital_index'] = column_name 
df_final

In [ ]:
# Closest hospital
hospital_id = []

# Looping through the final dataframe 
# and updating the corresponding hospital ids
for item, row in df_final.iterrows():
    
    hospital_id.append(df_hospital.loc[row['hospital_index'], 'id'])

# Updating the final dataframe
df_final['Hospital_id'] = hospital_id
df_final['Distance_to_hospital'] = min_dist
df_final

In [ ]:
df_final = df_final.drop(['hospital_index'], axis = 1)

# Supermarket
- Converting the latitude and longitude values to float datatype
- Calculate distance of each property location with each of the supermarkets
- Then calculate which is the minimum distance and its indices
- Loop through df_supermarket dataframe and use the above mentioned indices to fetch the 'Supermarket_id'
- Update the final dataframe with the new values found

In [ ]:
# Converting the string latitude and longitude
# to float datatype
df_supermarket['lat'] = df_supermarket.lat.astype(float)
df_supermarket['lng'] = df_supermarket.lng.astype(float)

In [ ]:

# List of lists of distances
main_dist_list = []

# Looping through final dataframe to calculate 
# distances from each property to each of the 
# Supermarkets
for item, row in df_final.iterrows():
    dist_list = []
    for item1, row1 in df_supermarket.iterrows():
        dist_list.append(haversine_fun(
            row['lat'],row['lng'], 
            row1['lat'], row1['lng']))
        
    main_dist_list.append(dist_list)
main_dist_list


In [ ]:
# Caculating the minimum distances from supermarket 
# to the property 

new = [] 

# List contains the minimum distance for all the rows
min_dist = [] 
for i in main_dist_list:
    new.append(i.index(min(i)))
    
    # Rounding off the distance to 4 decimals
    min_dist.append(round(min(i),4)) 
column_name = np.array(new) 

# Creating a new column which will contain the 
# corresponding indices of supermarkets

df_final['supermarket_index'] = column_name 
df_final

In [ ]:
# Updating the final dataframe with the appropriate supermarket id
# and their minimum distances from the properties

# Closest supermarket
supermarket_id = []
for item, row in df_final.iterrows():
    
    supermarket_id.append(df_supermarket.loc[row['supermarket_index'], 'id'])

df_final['Supermarket_id'] = supermarket_id
df_final['Distance_to_supermaket'] = min_dist
df_final

In [ ]:
df_final = df_final.drop(['supermarket_index'], axis = 1)

# Active train service on all weekdays
Check which service id has trains on all the weekdays (Monday-Friday)


In [ ]:
# Calender
df_gtfs_calendar = pd.read_csv('calendar.txt', sep=",", header=None)
df_gtfs_calendar.columns = ["service_id","monday","tuesday","wednesday","thursday","friday","saturday","sunday","start_date","end_date"]
df_gtfs_calendar = df_gtfs_calendar.iloc[1:]
df_gtfs_calendar

The service id T0 has active trains from Monday-Friday

# Travel Minutes to CBD
Calculating the average time taken to reach the Flinders Station from each of the station that is nearest to the property.

* Define the time range in which the trains must be monitored 
* For each of the station id nearest to each of the property check:
    - for each station id, filter out all the trip ids that stop in that particular station
    - filter out the dataframe to see the departure times between 7am and 9am
    - on this dataframe, check if all the trips belong to 'T0' service
    - check if each of the trip ids go to Flinders station
        - If they do: take the difference between the times when the trip_id arrived at Flinders station and at what time did the trip_id depart from the nearest station
        - If there is any element in the list, then take the average of that list and make a note of it
        - Make a note of the train station for which the average is being calculated for. 
           * Also, check which all stations are just one stop_sequence before the Flinders station (For Transfer_flag column)
        - If not: continue with the loop



In [ ]:
import datetime
# Convert the departure time column to time format
df_gtfs_stop_times['departure_time'] = pd.to_datetime(df_gtfs_stop_times['departure_time'],format= '%H:%M:%S').dt.time

# Range of time in which the logic has to be run (7am to 9am)
start_date = datetime.datetime.strptime('07:00:00', '%H:%M:%S').time()
end_date = datetime.datetime.strptime('09:00:00', '%H:%M:%S').time()

# Extending the range of arrival time to 12pm
end_date_1 = datetime.datetime.strptime('11:30:00', '%H:%M:%S').time()

In [ ]:

# empty lists to hold all the values as we go along
main_time = []
main_station = []
no_transfer_list = []

# Loop through the unique station ids from the final dataframe
for i in np.unique(df_final['Train_station_id'].values):
    travel_time = [] # List that holds all the travel time from the nearest train station
    
    # A temp dataframe that holds the filtered data for the station id
    temp = df_gtfs_stop_times[df_gtfs_stop_times['stop_id'] == i]
    
    # Limiting the departure time to range between 7am and 9am
    between_7_9_1 = temp.loc[(temp['departure_time'] >= start_date) & (temp['departure_time'] <= end_date)]
    
    # Filtering out all the 'T0' services in the above dataframe
    between_7_9_1['service'] = between_7_9_1['trip_id'].apply(lambda x: x.split('.')[1])
    between_7_9_1 = between_7_9_1[between_7_9_1['service'] == 'T0']
    
    # Looping though each of the trip_ids that depart between 7am and 9am and has service as 'T0'
    for item, row in between_7_9_1.iterrows():
        
        # Another temp dataframe to hold all the trip_id filtered rows
        temp1 = df_gtfs_stop_times[df_gtfs_stop_times['trip_id'] == row['trip_id']]   
        
        # Limiting the arrival time to range between 7am and 11:30pm
        arrive_7_9 = temp1.loc[(temp1['departure_time'] >= start_date) & (temp1['departure_time'] <= end_date_1)] 
        
        # Making the string values, float.
        # Arranging the dataframe in ascending order based on stop_sequence
        arrive_7_9['stop_sequence'] = arrive_7_9['stop_sequence'].astype(int)
        arrive_7_9 = arrive_7_9.sort_values(by=['stop_sequence'])
        
        # Check if Flinder's station id falls in the filtered dataframe
        if '19854' in arrive_7_9['stop_id'].values:
            
            # Make the 'stop_id' as the index of this dataframe
            arrive_7_9 = arrive_7_9.set_index('stop_id')
            
            # Convert the arrival time column from string to time format
            arr_time = datetime.datetime.strptime(arrive_7_9.loc['19854', 'arrival_time'], '%H:%M:%S').time()
            
            # Take depature time value in a variable
            dep_time = arrive_7_9.loc[i, 'departure_time']
            
            # Make the dates of the arrival and departure times to 'today'
            arr_time = datetime.datetime.combine(datetime.date.today(), arr_time)
            dep_time = datetime.datetime.combine(datetime.date.today(), dep_time)
            
            if arr_time > dep_time:
                diff_times = arr_time - dep_time 
            else:
                diff_times = dep_time - arr_time
            
            # Convert the time difference to minutes and update the time list
            minutes = diff_times.total_seconds() / 60
            travel_time.append(minutes)
                 
            # To check if there is a station just one stop before Flinders Station
            if arrive_7_9.loc['19854', 'stop_sequence'] - arrive_7_9.loc[i, 'stop_sequence'] == 1:
                no_transfer_list.append(i)
                
    # Check if there is any time to calculate the average on        
    if len(travel_time) > 0:
        
        # Update the list with average time
        main_time.append(round(sum(travel_time)/len(travel_time)))
        main_station.append(i)
        
        
# Refer: https://stackoverflow.com/questions/43305577/python-calculate-the-difference-between-two-datetime-time-objects/43308104

In [ ]:
# Create a dataframe to hold all the station id and their 
# corresponding minutes to travel

minutes_from_station = pd.DataFrame(columns=['station_id', 'time'])
minutes_from_station['station_id'] = main_station
minutes_from_station['time'] = main_time

# # Reset the index of the final Dataframe
# df_final = df_final.reset_index()

# Defualt value of 'travel_min_to_CBD'
df_final['travel_min_to_CBD'] = 0

df_final_copy = df_final.copy()

# loop through each row of final dataframe and update the travel
# min to CBD
for item, row in df_final_copy.iterrows():
    for item1, row1 in minutes_from_station.iterrows():
        if row['Train_station_id'] == row1['station_id']:
            df_final.loc[item, 'travel_min_to_CBD'] = row1['time']



In [ ]:
# Default value of transfer flag is -1
df_final['Transfer_flag'] = -1

# Loop through the final dataframe and assign 1 or 0 for transfer flag column 
for i, r in df_final_copy.iterrows():
    if r['Train_station_id'] in minutes_from_station['station_id'].values:
        df_final.loc[i, 'Transfer_flag'] = 1
        
    if r['Train_station_id'] == no_transfer_list[0]:
        df_final.loc[i, 'Transfer_flag'] = 0

In [ ]:
df_final

# Suburb
* Read the shapes datafiles into a dataframe
* Create a function to calculate the polygon coordinates and retrieve the suburb name
    - Check if the point object lies within the polygon provided


In [ ]:
import geopandas

shape_file = geopandas.read_file("VIC_LOCALITY_POLYGON_shp.shp")
shape_file

In [ ]:
# Extracting the suburb names
def suburb_names(lon, lat, loca_geo):
    point = Point(lon, lat)
    for key, value in loca_geo.items():
        p = Polygon(value)
        if p.contains(point):
            return key
    return 'not available'

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

# Default value of suburb coulmn is 'not available'
df_final['suburb'] = 'not available'

# Forming a dict from shape_file 
loca_geo = dict(zip(shape_file['VIC_LOCA_2'].values, shape_file['geometry'].values))

# Updating the suburb column in the final dataframe
df_final['suburb'] = df_final.apply(lambda geo: suburb_names(geo['lng'], geo['lat'], loca_geo), axis= 1)


df_final.head()

In [ ]:
df_final

In [ ]:
# Renaming property_id with Property_id
df_final.columns=[it.replace('property_id','Property_id') for it in df_final.columns]


# Reordering the dataframe
df_final = df_final[['Property_id', 'lat', 'lng', 'addr_street', 'suburb', 'price', 'property_type', 'year', 'bedrooms', 'bathrooms', 'parking_space', 'Shopping_center_id', 'Distance_to_sc', 'Train_station_id', 'Distance_to_train_station', 'travel_min_to_CBD', 'Transfer_flag', 'Hospital_id', 'Distance_to_hospital','Supermarket_id', 'Distance_to_supermaket' ]]


In [ ]:
# Write it to csv
df_final.to_csv('31009751_A3_solution.csv', index = False)

# Conclusion

A final csv is written with 21 columns as asked in the specifications. 

### Reading the remaining files which are not used in this code

In [ ]:
# Shapes
df_gtfs_shapes = pd.read_csv('shapes.txt', sep=",", header=None)
df_gtfs_shapes.columns = ["shape_id", "shape_pt_lat", "shape_pt_lon", "shape_pt_sequence","shape_dist_traveled"]
df_gtfs_shapes = df_gtfs_shapes.iloc[1:]
df_gtfs_shapes

In [ ]:
# Routes
df_gtfs_routes = pd.read_csv('routes.txt', sep=",", header=None)
df_gtfs_routes.columns = ["route_id","agency_id","route_short_name","route_long_name","route_type"]
df_gtfs_routes = df_gtfs_routes.iloc[1:]
df_gtfs_routes

In [ ]:
# Calender dates
df_gtfs_calendar_dates = pd.read_csv('calendar_dates.txt', sep=",", header=None)
df_gtfs_calendar_dates.columns = ["service_id","date","exception_type"]
df_gtfs_calendar_dates = df_gtfs_calendar_dates.iloc[1:]
df_gtfs_calendar_dates

In [ ]:
# Calender
df_gtfs_calendar = pd.read_csv('calendar.txt', sep=",", header=None)
df_gtfs_calendar.columns = ["service_id","monday","tuesday","wednesday","thursday","friday","saturday","sunday","start_date","end_date"]
df_gtfs_calendar = df_gtfs_calendar.iloc[1:]
df_gtfs_calendar

In [ ]:
# Agency
df_gtfs_agency = pd.read_csv('agency.txt', sep=",", header=None)
df_gtfs_agency.columns = ["agency_id","agency_name","agency_url","agency_timezone","agency_lang"]
df_gtfs_agency = df_gtfs_agency.iloc[1:]
df_gtfs_agency

In [ ]:
df_final

# Task 2 - Data Reshaping
Observing the effect of different normalization/transformation methods on the “price” ,
“Distance_to_sc”, “travel_min_to_CBD” , and “Distance_to_hospital” attributes. Following are the different methods used:

1. Standardization
2. Minmax normalization 
3. log transformation 
4. power transformation 
5. box-cox transformation

### Standardisation
Standardized values (also called standard scores or normal deviates) are the same thing as z-scores. So, it follows the formula: 

$$z = \frac{x - mean(x)}{\sigma}$$

This makes sure that each column that we are interested in has values around the value '0'.

Also, this is used to make all the column's unit values standard. For eg., we want to predict the 'price' (unit: Dollars) having distance to shopping center (unit: Kilometer), distance to hospital (unit: Kilometer) and travel minute to CBD (unit: minutes). To bring all the columns to a standard form, irrespective of the units that each of them hold, standardization is carried out. 

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.preprocessing import PowerTransformer

In [ ]:
df = pd.DataFrame(columns = ['price', 'Dist_sc', 'travel', 'Dist_hospital'])
df['price'] = df_final['price'].values.astype(float)
df['Dist_sc'] = df_final['Distance_to_sc'].values.astype(float)
df['travel'] = df_final['travel_min_to_CBD'].values.astype(float)
df['Dist_hospital'] = df_final['Distance_to_hospital'].values.astype(float)

### Before standardizing the columns

In [ ]:
df['price'].hist()

In [ ]:
df['Dist_hospital'].hist()

In [ ]:
df['travel'].hist()

In [ ]:


names = df.columns

# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_df, columns=names)


### After standardizing the columns
* Observe the x axis values after standardizing the column. The values are in and around '0'

In [ ]:
scaled_df['price'].hist()

In [ ]:
scaled_df['Dist_sc'].hist()

In [ ]:
scaled_df['Dist_hospital'].hist()

In [ ]:
scaled_df['travel'].hist()

# Predicting the price for standardized columns

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Linear regression for standardized dataset
dependent = scaled_df['price']
independent = scaled_df[['Dist_sc', 'travel','Dist_hospital']]

# Using Linear Regression Model from Sklearn 
linear_regression = LinearRegression()
linear_regression.fit(independent, dependent)

std_predict = linear_regression.predict(independent)

In [ ]:
scaled_df['price']

# MIN/MAX

Min-max normalization is one of the most common ways to normalize data. For every feature, the minimum value of that feature gets transformed into a 0, the maximum value gets transformed into a 1, and every other value gets transformed into a decimal between 0 and 1.

In [ ]:
# min max scaler object
minMax_transform = preprocessing.MinMaxScaler()

# Transforming all the required columns
price = minMax_transform.fit_transform(df[['price']])
Dist_sc = minMax_transform.fit_transform(df[['Dist_sc']])
travel = minMax_transform.fit_transform(df[['travel']])
Dist_hospital = minMax_transform.fit_transform(df[['Dist_hospital']])

# Updating the dataframe with transformed values
df_final['price_minmax'] = pd.DataFrame(price)
df_final['Distance_to_sc_minmax'] = pd.DataFrame(Dist_sc)
df_final['travel_min_to_CBD_minmax'] = pd.DataFrame(travel)
df_final['Distance_to_hospital_minmax'] = pd.DataFrame(Dist_hospital)

df_final

### After Min/Max Normalization

All the values are scaled between 0 and 1. 0 being the minimum in of that column, 1 being maximum of that column, 0.5 being halfway through in that column

In [ ]:
df_final['price_minmax'].hist()

In [ ]:
df_final['Distance_to_sc_minmax'].hist()

In [ ]:
df_final['travel_min_to_CBD_minmax'].hist()

In [ ]:
df_final['Distance_to_hospital_minmax'].hist()

In [ ]:
# Linear regression for standardized dataset
dependent = df_final['price_minmax']
independent = df_final[['Distance_to_sc_minmax', 'travel_min_to_CBD_minmax','Distance_to_hospital_minmax']]

# Using Linear Regression Model from Sklearn 
linear_regression = LinearRegression()
linear_regression.fit(independent, dependent)


In [ ]:
minmax_predict = linear_regression.predict(independent)
minmax_predict

# Log transformation
* Taking log of each column and then predicting the values
* Since there are 0 values in travel_min_to_CBD column, taking log of 0 will result in '-inf' . Hence impute '0' with '0.001' values

In [ ]:
# Calculate natural logarithm on the log transformation 
df_final['price'] = df_final[['price']].values.astype(float)
df_final['Distance_to_sc'] = df_final[['Distance_to_sc']].values.astype(float)
df_final['travel_min_to_CBD'] = df_final[['travel_min_to_CBD']].values.astype(float)
df_final['Distance_to_hospital'] = df_final[['Distance_to_hospital']].values.astype(float)

# Imputing 0's with 0.001 and then taking the log of the column
df_final['travel_min_to_CBD_log'] = df_final['travel_min_to_CBD'].values

df_final['travel_min_to_CBD_log'] = df_final['travel_min_to_CBD_log'].apply(lambda x: 0.001 if x == 0 else x)

df_final[['price_log']] = np.log(df_final['price'])
df_final[['Distance_to_sc_log']] = np.log(df_final['Distance_to_sc'])
df_final[['travel_min_to_CBD_log']] = np.log(df_final['travel_min_to_CBD_log'])
df_final[['Distance_to_hospital_log']] = np.log(df_final['Distance_to_hospital'])

df_final

### Histograms for each of the log transformed columns

In [ ]:
df_final['price_log'].hist()

In [ ]:
df_final['Distance_to_sc_log'].hist()

In [ ]:
df_final['travel_min_to_CBD_log'].hist()

In [ ]:
df_final['Distance_to_hospital_log'].hist()

In [ ]:
# Linear regression for standardized dataset
dependent = df_final['price_log']
independent = df_final[['Distance_to_sc_log', 'travel_min_to_CBD_log','Distance_to_hospital_log']]

# Using Linear Regression Model from Sklearn 
linear_regression = LinearRegression()
linear_regression.fit(independent, dependent)

log_predict = linear_regression.predict(independent)

In [ ]:
df_final['price_log']

## Power transformation (box-cox)
A Box Cox transformation is a transformation of a non-normal dependent variables into a normal shape. Here, since none of the values should be 0 or lesser than 0, I am imputing 0.001 inplace of 0.

Refer: https://www.statisticshowto.com/box-cox-transformation/#:~:text=What%20is%20a%20Box%20Cox,a%20broader%20number%20of%20tests.

In [ ]:
df = pd.DataFrame(columns = ['price', 'Dist_sc', 'travel', 'Dist_hospital'])
df['price'] = df_final['price']
df['Dist_sc'] = df_final['Distance_to_sc']
df['travel'] = df_final['travel_min_to_CBD']
df['Dist_hospital'] = df_final['Distance_to_hospital']

df['travel'] = df['travel'].apply(lambda x: 0.001 if x == 0 else x)

from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
power_transformer = PowerTransformer(method = 'box-cox')
power_transformer.fit(df)                      
train_power_transformer = power_transformer.transform(df)
transform_df = pd.DataFrame(train_power_transformer, columns = ['price', 'Dist_sc', 'travel', 'Dist_hospital'])
transform_df


It is a power transform that assumes the values of the input variable to which it is applied are strictly positive. That means 0 and negative values are not supported. It is important to note that the Box-Cox procedure can only be applied to data that is strictly positive.

Refer: https://machinelearningmastery.com/power-transforms-with-scikit-learn/#:~:text=Box%2DCox%20Transform,-The%20Box%2DCox&text=It%20is%20a%20power%20transform,data%20that%20is%20strictly%20positive.


In [ ]:
# Linear regression for standardized dataset
dependent = transform_df['price']
independent = transform_df[['Dist_sc', 'travel','Dist_hospital']]

# Using Linear Regression Model from Sklearn 
linear_regression = LinearRegression()
linear_regression.fit(independent, dependent)

bcox_predict = linear_regression.predict(independent)

In [ ]:
transform_df['price'].hist()

In [ ]:
transform_df['Dist_sc'].hist()

In [ ]:
transform_df['travel'].hist()

In [ ]:
transform_df['Dist_hospital'].hist()

# RMSE score 

##### 1. Standardized rmse

In [ ]:
from sklearn.metrics import mean_squared_error


In [ ]:
# RMSE for standardized 
mean_squared_error(scaled_df['price'].values, std_predict)

##### 2. Min/Max rmse

In [ ]:
# RMSE for min/max rmse
mean_squared_error(df_final['price_minmax'].values, minmax_predict)

##### 3. Log Transformation

In [ ]:
# RMSE for Log transformation

mean_squared_error(df_final['price_log'].values, log_predict)

##### 4. Box-cox Transformation

In [ ]:
# RMSE for Box-cox transformation

mean_squared_error(transform_df['price'].values, bcox_predict)

# Conclusion
Transformations in general, are applied to make the data more testable on different models. And inorder to evaluate which model is better than which, we need to know the RMSE scores for each models and narrow down to the one which has the least RMSE score.

From our observation, for this dataset, Min/Max transformation will give a better prediction than the rest of them since it has the least rmse score of 0.01617. 